In [1]:
import pandas as pd
import numpy as np

import statsmodels.tsa.api as smt
from statsmodels.tsa.seasonal import seasonal_decompose  # Decomposition
from stldecompose import decompose, forecast             # Decomposition & forecasting
from statsmodels.tsa.holtwinters import ExponentialSmoothing #ETS Models

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
sales_data = pd.read_csv("Sales-and-Marketing.csv")
sales_data.head(5)

,Time Period,Sales,Marketing Expense
0,Jan-11,397,486.64
1,Feb-11,400,501.80
2,Mar-11,498,437.09
3,Apr-11,536,565.16
4,May-11,596,744.15


In [6]:
sales_data.tail(5)

,Month-Year,Number of Tractor Sold
139,Aug-14,848
140,Sep-14,640
141,Oct-14,581
142,Nov-14,519
143,Dec-14,605


In [10]:
sales_data.rename(columns = {'Marketing Expense': 'MS'}, inplace=True)
sales_data.head(2)

,Time Period,Sales,MS
0,Jan-11,397,486.64
1,Feb-11,400,501.80


In [11]:
sales_data.tail(3)

,Time Period,Sales,MS
45,Oct-14,581,570.30
46,Nov-14,519,527.60
47,Dec-14,605,559.75


In [12]:
# since the complete date was not mentioned, we assume that it was the first of every month
dates = pd.date_range(start='2011-01-01', freq='MS', periods=len(sales_data))

In [13]:
dates

DatetimeIndex(['2011-01-01', '2011-02-01', '2011-03-01', '2011-04-01',
               '2011-05-01', '2011-06-01', '2011-07-01', '2011-08-01',
               '2011-09-01', '2011-10-01', '2011-11-01', '2011-12-01',
               '2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01',
               '2012-05-01', '2012-06-01', '2012-07-01', '2012-08-01',
               '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01',
               '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01',
               '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01',
               '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01',
               '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01',
               '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01',
               '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01'],
              dtype='datetime64[ns]', freq='MS')

In [14]:
sales_data.drop(['Time Period'], axis=1, inplace=True)
sales_data.head(2)

,Sales,MS
0,397,486.64
1,400,501.80


In [15]:
sales_data.rename(columns={'Sales':'Tractor-Sales'}, inplace=True)
sales_data.head(5)

,Tractor-Sales,MS
0,397,486.64
1,400,501.80
2,498,437.09
3,536,565.16
4,596,744.15


In [16]:
# Perform Dickey-Fuller test:
from statsmodels.tsa.stattools import adfuller
print('Results of Dickey-Fuller Test:')
dftest = adfuller(sales_data['Tractor-Sales'])
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

Results of Dickey-Fuller Test:
Test Statistic                  1.434334
p-value                         0.997265
#lags Used                     10.000000
Number of Observations Used    37.000000
Critical Value (1%)            -3.620918
Critical Value (5%)            -2.943539
Critical Value (10%)           -2.610400
dtype: float64


In [18]:
sales_data_diff = sales_data.diff(periods=1)
sales_data_diff.dropna(inplace=True)

In [19]:
# Perform Dickey-Fuller test:
from statsmodels.tsa.stattools import adfuller
print('Results of Dickey-Fuller Test:')
dftest = adfuller(sales_data_diff['Tractor-Sales'])
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

Results of Dickey-Fuller Test:
Test Statistic                -7.015650e+00
p-value                        6.743731e-10
#lags Used                     1.000000e+01
Number of Observations Used    3.600000e+01
Critical Value (1%)           -3.626652e+00
Critical Value (5%)           -2.945951e+00
Critical Value (10%)          -2.611671e+00
dtype: float64


In [20]:
sales_data['Lag_1'] = sales_data['MS'].shift(1)
sales_data['Lag_2'] = sales_data['MS'].shift(2)
sales_data['Lag_3'] = sales_data['MS'].shift(3)
sales_data['Lag_4'] = sales_data['MS'].shift(4)

In [22]:
sales_data.head(6)

,Tractor-Sales,MS,Lag_1,Lag_2,Lag_3,Lag_4
0,397,486.64,NaN,NaN,NaN,NaN
1,400,501.80,486.64,NaN,NaN,NaN
2,498,437.09,501.80,486.64,NaN,NaN
3,536,565.16,437.09,501.80,486.64,NaN
4,596,744.15,565.16,437.09,501.80,486.64
5,591,548.74,744.15,565.16,437.09,501.80
